# LlamaIndex Integration

> Natural language interface for SQLite database using LlamaIndex

In [ ]:
#| default_exp llamaindex_integration

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import sqlite3
from typing import Optional, Dict, Any, List
from llamaindex_mcp_nbdev.core import init_db, SQLiteMCPServer

# Check if LlamaIndex is installed
try:
    from llama_index.core import SQLDatabase, Settings
    from llama_index.core.query_engine import NLSQLTableQueryEngine
    from llama_index.llms.openai import OpenAI
    LLAMAINDEX_AVAILABLE = True
except ImportError:
    LLAMAINDEX_AVAILABLE = False
    print("LlamaIndex not installed. Install with: pip install llama-index")

## Basic Text-to-SQL Query Engine

In [ ]:
#| export

class LlamaIndexSQLServer(SQLiteMCPServer):
    """Extended MCP Server with LlamaIndex natural language query capabilities."""
    
    def __init__(self, server_name: str = 'llamaindex-sqlite', 
                 db_path: str = 'demo.db',
                 openai_api_key: Optional[str] = None):
        """Initialize server with LlamaIndex support.
        
        Args:
            server_name: Name of the MCP server
            db_path: Path to SQLite database
            openai_api_key: OpenAI API key for LLM queries (optional)
        """
        super().__init__(server_name, db_path)
        self.query_engine = None
        
        if LLAMAINDEX_AVAILABLE:
            self._setup_llamaindex(openai_api_key)
    
    def _setup_llamaindex(self, api_key: Optional[str] = None):
        """Setup LlamaIndex query engine."""
        try:
            # Set up LLM
            if api_key:
                os.environ["OPENAI_API_KEY"] = api_key
                Settings.llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
            
            # Create SQL database object
            sql_database = SQLDatabase.from_uri(f"sqlite:///{self.db_path}")
            
            # Create query engine
            self.query_engine = NLSQLTableQueryEngine(
                sql_database=sql_database,
                tables=["people"],  # Specify which tables to include
            )
            
            # Add natural language query tool
            self._add_nl_query_tool()
            
        except Exception as e:
            print(f"Warning: Could not setup LlamaIndex: {e}")
            print("Natural language queries will not be available.")
    
    def _add_nl_query_tool(self):
        """Add natural language query tool to MCP server."""
        
        @self.mcp.tool()
        def nl_query(question: str) -> Dict[str, Any]:
            """Ask questions about the database in natural language.
            
            Args:
                question: Natural language question about the data
                
            Returns:
                Dict containing the answer and generated SQL query
                
            Examples:
                - "What is the average age of developers?"
                - "How many people are older than 30?"
                - "List all engineers sorted by age"
                - "Who is the youngest person in the database?"
            """
            if not self.query_engine:
                return {
                    "error": "Natural language queries not available. Please set up LlamaIndex."
                }
            
            try:
                # Query using natural language
                response = self.query_engine.query(question)
                
                # Extract SQL query if available
                sql_query = getattr(response, 'sql_query', None)
                
                return {
                    "success": True,
                    "answer": str(response),
                    "sql_query": sql_query,
                    "metadata": response.metadata if hasattr(response, 'metadata') else {}
                }
                
            except Exception as e:
                return {
                    "success": False,
                    "error": str(e)
                }

## Query Examples and Capabilities

In [ ]:
#| export

def get_sample_queries() -> List[str]:
    """Get sample natural language queries for demonstration."""
    return [
        "What is the average age of all people?",
        "How many developers are in the database?",
        "Who is the oldest person?",
        "List all professions and their count",
        "Show me people younger than 30",
        "What's the age range in our database?",
        "Find all engineers sorted by age",
        "How many people are there in each age group (20s, 30s, 40s)?",
        "What is the most common profession?",
        "Show me the youngest developer"
    ]

## Standalone Query Function

In [ ]:
#| export

def query_database_nl(question: str, db_path: str = "demo.db") -> Dict[str, Any]:
    """Query SQLite database using natural language.
    
    Args:
        question: Natural language question
        db_path: Path to SQLite database
        
    Returns:
        Dictionary with answer and generated SQL
    """
    if not LLAMAINDEX_AVAILABLE:
        return {
            "error": "LlamaIndex not installed. Install with: pip install llama-index"
        }
    
    try:
        # Create SQL database connection
        sql_database = SQLDatabase.from_uri(f"sqlite:///{db_path}")
        
        # Create query engine
        query_engine = NLSQLTableQueryEngine(
            sql_database=sql_database,
            tables=["people"],
        )
        
        # Execute query
        response = query_engine.query(question)
        
        return {
            "success": True,
            "answer": str(response),
            "sql_query": getattr(response, 'sql_query', None)
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

## Demo Functions

In [ ]:
#| eval: false
# Test database setup
def setup_test_data(db_path: str = "test_llama.db"):
    """Create test database with sample data."""
    conn, cursor = init_db(db_path)
    
    # Clear existing data
    cursor.execute("DELETE FROM people")
    
    # Insert test data
    test_data = [
        ("Alice Johnson", 28, "Developer"),
        ("Bob Smith", 35, "Engineer"),
        ("Carol Davis", 42, "Manager"),
        ("David Wilson", 23, "Developer"),
        ("Eve Brown", 31, "Designer"),
        ("Frank Miller", 45, "Engineer"),
        ("Grace Lee", 29, "Developer"),
        ("Henry Taylor", 38, "Manager"),
        ("Iris Chen", 26, "Designer"),
        ("Jack White", 52, "Engineer")
    ]
    
    cursor.executemany(
        "INSERT INTO people (name, age, profession) VALUES (?, ?, ?)",
        test_data
    )
    
    conn.commit()
    conn.close()
    
    print(f"Test database created with {len(test_data)} records")

# Create test database
setup_test_data()

In [ ]:
#| eval: false
# Demonstrate natural language queries (requires API key)
if LLAMAINDEX_AVAILABLE:
    print("LlamaIndex is available!")
    print("\nTo use natural language queries, you need to:")
    print("1. Set OPENAI_API_KEY environment variable")
    print("2. Or pass openai_api_key parameter when creating server")
    print("\nExample queries you can ask:")
    for query in get_sample_queries()[:5]:
        print(f"- {query}")
else:
    print("LlamaIndex not installed. Install with:")
    print("pip install llama-index llama-index-llms-openai")

## Usage Examples

In [ ]:
#| export

def main():
    """Run LlamaIndex-enabled MCP server."""
    import argparse
    
    parser = argparse.ArgumentParser(description="SQLite MCP Server with LlamaIndex")
    parser.add_argument(
        "--server_type", 
        type=str, 
        default="sse",
        choices=["sse", "stdio"],
        help="Type of server to run"
    )
    parser.add_argument(
        "--db_path",
        type=str,
        default="demo.db",
        help="Path to SQLite database file"
    )
    parser.add_argument(
        "--openai_api_key",
        type=str,
        default=None,
        help="OpenAI API key for natural language queries"
    )
    
    args = parser.parse_args()
    
    # Create and run server
    server = LlamaIndexSQLServer(
        db_path=args.db_path,
        openai_api_key=args.openai_api_key
    )
    
    print("Starting LlamaIndex-enabled SQLite MCP Server...")
    if LLAMAINDEX_AVAILABLE and server.query_engine:
        print("Natural language queries are enabled!")
        print("Available tools: add_data, read_data, nl_query")
    else:
        print("Running in basic mode (SQL only)")
        print("Available tools: add_data, read_data")
    
    server.run(args.server_type)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()